# Graded Exercise 1: Extraction of Discriminative Features for Anomaly Detection on Acoustic Data

- **Course**: [CIVIL-426 - Machine Learning for Predictive Maintenance](https://edu.epfl.ch/coursebook/en/machine-learning-for-predictive-maintenance-applications-CIVIL-426)
- **Start Date**: 2024.09.19 at 10h15
- **Due Date**: 2024.10.02 at 23h59
- **Student 0**:
    - First and Last name: `FILL HERE`
    - SCIPER: `FILL HERE`
- **Student 1**:
    - First and Last name: `FILL HERE`
    - SCIPER: `FILL HERE`

## Introduction

This notebook is divided in 2 parts:
- [Part 0](#part-0-demonstration): Demonstration on a toy dataset

This part shows how to extract discriminative features for anomaly detection on a toy 
dataset. This part of the notebook runs without any issues on a properly configured 
machine. You are **not** expected to fill anything in this part.

- [Part 1](#part-1-exercise): Graded exercise

This part contains the detailed questions and assignments of the graded exercise. You
are expected to fill this part with your own code and your own answers to the questions.

⚠️ The deadline to hand-in a PDF report along this complete notebook with 
[Part 1](#part-1-exercise) filled is **October 2nd, 2024 at 23h59**. No submissions 
will be accepted past this deadline. ⚠️

---

## Part 0: Demonstration

### 0. Introduction

The **goals** of this exercise are:

* Extracting statistical features of acoustic signals to discriminate between normal and abnormal samples
* Defining a threshold-based anomaly detection rule based on normal samples only

The **requirements** are:
* Your method can use a single feature or a combination of features
* You must include visualization in a meaningful way
* Your method must be developed based on the **TRAIN set** only

This part contains an example on a toy dataset.

1. **Train**: training set containing only healthy (normal) samples
2. **Test**: test set containing both normal and abnormal samples.

### 1. Notebook Configuration

In [ ]:
# Packages for visualization
%pip install seaborn matplotlib-venn --upgrade --quiet
# Packages for feature extraction and machine learning
%pip install librosa numpy pandas scipy --upgrade --quiet


In [2]:
import librosa
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn3
import numpy as np
from numpy.fft import fft
import pandas as pd
import seaborn as sns
import scipy.stats as scist

# Matplotlib configuration for notebooks
%matplotlib inline


### 2. Load the Dataset 

In [3]:
# Load the dataset from your disk (you must have downloaded the file)
data = np.load("toy_data.npz")

# Extract the training and testing set
train_set = data["train"]
test_set = data["test"]

train_set_size = train_set.shape[0]
train_set_timesteps = train_set.shape[1]
test_set_size = test_set.shape[0]
test_set_timesteps = test_set.shape[1]


In [ ]:
# About the train dataset
print(
    f"The TRAINING set contains {train_set_size} recordings. "
    f"Each recording contains {train_set_timesteps} samples."
)

# About the test dataset
print(
    f"The TESTING set contains {test_set_size} recordings. "
    f"Each recording contains {test_set_timesteps} samples."
)


### 3. Visual Signals

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[0].plot(train_set[0])
ax[0].set_title("TRAIN set Item 0")
ax[1].plot(test_set[0])
ax[1].set_title("TEST set Item 0")
ax[2].plot(test_set[1])
ax[2].set_title("TEST set Item 1")

for i in range(3):
    ax[i].set_xlabel("Sample")
    ax[i].set_ylabel("Signal Intensity")


### 4. Compute Absolute Values

In [6]:
train_set_abs = np.abs(train_set)
test_set_abs = np.abs(test_set)


### 5. Compute Min, Max, Centered Moments and Centralized Moments

In [7]:
# Compute moments 1 to 4 on the training set and assign them in a Pandas dataframe
train_moments = pd.DataFrame(
    {
        "MaxAbs": np.amax(train_set_abs, axis=1),
        "Mean": np.mean(train_set, axis=1),
        "Variance": scist.moment(train_set, moment=2, axis=1, nan_policy="propagate"),
        "Skewness": scist.skew(train_set, axis=1, bias=True, nan_policy="propagate"),
        "Kurtosis": scist.kurtosis(
            train_set, axis=1, fisher=False, bias=True, nan_policy="propagate"
        ),
        "type": "train",
    }
)

# Compute moments 1 to 4 on the testing set and assign them in a Pandas dataframe
test_moments = pd.DataFrame(
    {
        "MaxAbs": np.amax(test_set_abs, axis=1),
        "Mean": np.mean(test_set, axis=1),
        "Variance": scist.moment(test_set, moment=2, axis=1, nan_policy="propagate"),
        "Skewness": scist.skew(test_set, axis=1, bias=True, nan_policy="propagate"),
        "Kurtosis": scist.kurtosis(
            test_set, axis=1, fisher=False, bias=True, nan_policy="propagate"
        ),
        "type": "test",
    }
)


### 6. Fourier Transform and Spectrum Skewness

In [8]:
# Remove DC compoent and calculate FFT
train_spectrum = 2.0 / train_set_size * np.abs(fft(train_set, axis=1, norm=None))
test_spectrum = 2.0 / test_set_size * np.abs(fft(test_set, axis=1, norm=None))

# Compute Skewness of the Sprectrum excluding the 0 [Hz] component
train_moments["SpSkew"] = scist.skew(
    train_spectrum[:, 1:], axis=1, bias=True, nan_policy="propagate"
)
test_moments["SpSkew"] = scist.skew(
    test_spectrum[:, 1:], axis=1, bias=True, nan_policy="propagate"
)


In [ ]:
# If working on macOS, you want to specify that the diagonal plot should be histograms to avoid errors when plotting
# Otherwise you can leave the lines commented
sns.pairplot(
    train_moments,
    vars=[c for c in train_moments.columns if c != "type"],
    height=3,
    plot_kws={"alpha": 0.4, "s": 40, "edgecolor": None},
)
#            diag_kind = 'hist',
#           diag_kws = {'alpha': 0.4, 'log':True, 'density':True}
# )


In [ ]:
# If working on macOS, you want to specify that the diagonal plot should be histograms to avoid errors when plotting
# Otherwise you can leave the lines commented
sns.pairplot(
    test_moments,

    vars=[c for c in test_moments.columns if c != "type"],

    height=3,

    plot_kws={"alpha": 0.4, "s": 40, "edgecolor": None},
)
# ,
#            diag_kind = 'hist',
#           diag_kws = {'alpha': 0.4, 'log':True, 'density':True})


### 7. Defining thresholds based on normal data percentiles

Here, we define an anomaly detection rule based on thresholds, corresponding to the [0.5, 99.5] percentiles of the feature distributions of normal samples. In other words, a feature higher than 99.5% or lower than 0.5% of the normal features is considered an anomaly. Using only normal samples to base the decision is motivated by the facts that:

1. Few anomalies are observed
2. We cannot observe every possible anomaly

Anomalies are outliers with respect to the normal data distribution.

In [ ]:
quantiles = [0.5, 99.5]
limit_max_abs = np.percentile(train_moments.loc[:, "MaxAbs"], quantiles)
limit_mean = np.percentile(train_moments.loc[:, "Mean"], quantiles)
limit_variance = np.percentile(train_moments.loc[:, "Variance"], quantiles)
lim_skew = np.percentile(train_moments.loc[:, "Skewness"], quantiles)
limit_kurtosis = np.percentile(train_moments.loc[:, "Kurtosis"], quantiles)
limit_sp_skew = np.percentile(train_moments.loc[:, "SpSkew"], quantiles)

train_percentage = np.linspace(0, 100, train_set_size)  # From 0% to 100%

plt.figure(1, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Fig.1: Percentiles of the 4 first centered Moment for the training set.",
    fontsize=20,
)
plt.subplot(2, 2, 1)
plt.plot(
    np.sort(train_moments.loc[:, "Mean"]), train_percentage, marker=".", linewidth=0
)
plt.plot([limit_mean[0], limit_mean[0]], plt.ylim(), color="k")
plt.plot([limit_mean[1], limit_mean[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Mean-percentile")
plt.ylabel("Percentile")
plt.subplot(2, 2, 2)
plt.plot(
    np.sort(train_moments.loc[:, "Variance"]), train_percentage, marker=".", linewidth=0
)
plt.plot([limit_variance[0], limit_variance[0]], plt.ylim(), color="k")
plt.plot([limit_variance[1], limit_variance[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Variance-percentile")
plt.ylabel("Percentile")
plt.subplot(2, 2, 3)
plt.plot(
    np.sort(train_moments.loc[:, "Skewness"]), train_percentage, marker=".", linewidth=0
)
plt.plot([lim_skew[0], lim_skew[0]], plt.ylim(), color="k")
plt.plot([lim_skew[1], lim_skew[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Skewness-percentile")
plt.ylabel("Percentile")
plt.subplot(2, 2, 4)
plt.plot(
    np.sort(train_moments.loc[:, "Kurtosis"]), train_percentage, marker=".", linewidth=0
)
plt.plot([limit_kurtosis[0], limit_kurtosis[0]], plt.ylim(), color="k")
plt.plot([limit_kurtosis[1], limit_kurtosis[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Kurtosis-percentile")
plt.ylabel("Percentile")


In [ ]:
plt.figure(1, figsize=(15, 8))
plt.clf()
plt.suptitle("Fig.2: Percentiles of MaxAbs and SpSkew", fontsize=20)
plt.subplot(1, 2, 1)
plt.plot(np.sort(train_moments.loc[:, "MaxAbs"]), train_percentage, marker=".", linewidth=0)
plt.plot([limit_max_abs[0], limit_max_abs[0]], plt.ylim(), color="k")
plt.plot([limit_max_abs[1], limit_max_abs[1]], plt.ylim(), color="k")
plt.xlabel("Value of the MaxAbs-percentile")
plt.ylabel("Percentile")
plt.subplot(1, 2, 2)
plt.plot(np.sort(train_moments.loc[:, "SpSkew"]), train_percentage, marker=".", linewidth=0)
plt.plot([limit_sp_skew[0], limit_sp_skew[0]], plt.ylim(), color="k")
plt.plot([limit_sp_skew[1], limit_sp_skew[1]], plt.ylim(), color="k")
plt.xlabel("Value of the SpSkew-percentile")
plt.ylabel("Percentile")


In [ ]:
quantiles = [0.2, 99.8]
limit_max_abs = np.percentile(train_moments.loc[:, "MaxAbs"], quantiles)
limit_mean = np.percentile(train_moments.loc[:, "Mean"], quantiles)
limit_variance = np.percentile(train_moments.loc[:, "Variance"], quantiles)
lim_skew = np.percentile(train_moments.loc[:, "Skewness"], quantiles)
limit_kurtosis = np.percentile(train_moments.loc[:, "Kurtosis"], quantiles)
limit_sp_skew = np.percentile(train_moments.loc[:, "SpSkew"], quantiles)

test_percentage = np.linspace(0, 100, test_set_size)  # from 0% to 100% with step 100/nTrain

plt.figure(2, figsize=(15, 10))
plt.clf()
plt.suptitle(
    "Fig.3: Percentiles of the 4 first centered moments "
    + "for both training and test sets with proposed thresholds.",
    fontsize=20,
)
plt.subplot(3, 2, 1)
plt.plot(np.sort(train_moments.loc[:, "Mean"]), train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(test_moments.loc[:, "Mean"]), test_percentage, marker=".", linewidth=0)
plt.plot([limit_mean[0], limit_mean[0]], plt.ylim(), color="k")
plt.plot([limit_mean[1], limit_mean[1]], plt.ylim(), color="k")
plt.xlabel("Mean")
plt.ylabel("Percentile")

plt.subplot(3, 2, 2)
plt.plot(np.sort(train_moments.loc[:, "Variance"]), train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(test_moments.loc[:, "Variance"]), test_percentage, marker=".", linewidth=0)
plt.plot([limit_variance[0], limit_variance[0]], plt.ylim(), color="k")
plt.plot([limit_variance[1], limit_variance[1]], plt.ylim(), color="k")
plt.xlabel("Variance")
plt.ylabel("Percentile")

plt.subplot(3, 2, 3)
plt.plot(np.sort(train_moments.loc[:, "Skewness"]), train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(test_moments.loc[:, "Skewness"]), test_percentage, marker=".", linewidth=0)
plt.plot([lim_skew[0], lim_skew[0]], plt.ylim(), color="k")
plt.plot([lim_skew[1], lim_skew[1]], plt.ylim(), color="k")
plt.xlabel("Skewness")
plt.ylabel("Percentile")
# plt.xlim([-50,50])

plt.subplot(3, 2, 4)
plt.plot(np.sort(train_moments.loc[:, "Kurtosis"]), train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(test_moments.loc[:, "Kurtosis"]), test_percentage, marker=".", linewidth=0)
plt.plot([limit_kurtosis[0], limit_kurtosis[0]], plt.ylim(), color="k")
plt.plot([limit_kurtosis[1], limit_kurtosis[1]], plt.ylim(), color="k")
plt.xlabel("Kurtosis")
plt.ylabel("Percentile")
# plt.xlim([-50,5000])
plt.figlegend(
    ["Train", "Test", "Threshold"], loc="lower center", ncol=5, labelspacing=0.0
)

plt.subplot(3, 2, 5)
plt.plot(np.sort(train_moments.loc[:, "MaxAbs"]), train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(test_moments.loc[:, "MaxAbs"]), test_percentage, marker=".", linewidth=0)
plt.plot([limit_max_abs[0], limit_max_abs[0]], plt.ylim(), color="k")
plt.plot([limit_max_abs[1], limit_max_abs[1]], plt.ylim(), color="k")
plt.xlabel("MaxAbs")
plt.ylabel("Percentile")

plt.subplot(3, 2, 6)
plt.plot(np.sort(train_moments.loc[:, "SpSkew"]), train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(test_moments.loc[:, "SpSkew"]), test_percentage, marker=".", linewidth=0)
plt.plot([limit_sp_skew[0], limit_sp_skew[0]], plt.ylim(), color="k")
plt.plot([limit_sp_skew[1], limit_sp_skew[1]], plt.ylim(), color="k")
plt.xlabel("SpSkew")
plt.ylabel("Percentile")


In [ ]:
# select points either below 0.5 percentile or above 99.5 percentile of each moments and min-max limits
abn_train_mean = np.where(
    (train_moments.loc[:, "Mean"] < limit_mean[0])
    | (train_moments.loc[:, "Mean"] > limit_mean[1])
)[0]
abn_train_variance = np.where(
    (train_moments.loc[:, "Variance"] < limit_variance[0])
    | (train_moments.loc[:, "Variance"] > limit_variance[1])
)[0]
abn_train_skew = np.where(
    (train_moments.loc[:, "Skewness"] < lim_skew[0])
    | (train_moments.loc[:, "Skewness"] > lim_skew[1])
)[0]
abn_train_kurtosis = np.where(
    (train_moments.loc[:, "Kurtosis"] < limit_kurtosis[0])
    | (train_moments.loc[:, "Kurtosis"] > limit_kurtosis[1])
)[0]

abn_train_max_abs = np.where(
    (train_moments.loc[:, "MaxAbs"] < limit_max_abs[0])
    | (train_moments.loc[:, "MaxAbs"] > limit_max_abs[1])
)[0]
abn_train_sp_skew = np.where(
    (train_moments.loc[:, "SpSkew"] < limit_sp_skew[0])
    | (train_moments.loc[:, "SpSkew"] > limit_sp_skew[1])
)[0]

all_abnormalities_train = np.unique(
    np.concatenate(
        (
            abn_train_mean,
            abn_train_variance,
            abn_train_skew,
            abn_train_kurtosis,
            abn_train_max_abs,
            abn_train_sp_skew,
        )
    )
)

# Compare sets together. Loop over each pair of sets and compute how many IDs they have in common
train_dict = {
    "Mean": abn_train_mean,
    "Variance": abn_train_variance,
    "Skewness": abn_train_skew,
    "Kurtsosis": abn_train_kurtosis,
    "MaxAbs": abn_train_max_abs,
    "SpSkew": abn_train_sp_skew,
}
train_dict_keys = list(train_dict.keys())

print("TRAINING DATASET")
for i in range(train_dict.__len__()):
    k1 = train_dict_keys[i]
    print("{}: {} anomalies found.".format(k1, train_dict[k1].__len__()))

anomalies_count_train = np.array(
    [np.sum([i in train_dict[k] for k in train_dict]) for i in range(train_set_size)]
)
for i in range(1, 5):
    print(
        "{} samples are found as anomalous by at least {} selection methods.".format(
            len(np.where(anomalies_count_train >= i)[0]), i
        )
    )

######################################################################################################

# select points either below 0.2 percentile or above 99.8 percentile of each moments and min-max limits
abn_test_mean = np.where(
    (test_moments.loc[:, "Mean"] < limit_mean[0])
    | (test_moments.loc[:, "Mean"] > limit_mean[1])
)[0]
abn_test_variance = np.where(
    (test_moments.loc[:, "Variance"] < limit_variance[0])
    | (test_moments.loc[:, "Variance"] > limit_variance[1])
)[0]
abn_test_skew = np.where(
    (test_moments.loc[:, "Skewness"] < lim_skew[0])
    | (test_moments.loc[:, "Skewness"] > lim_skew[1])
)[0]
abn_test_kurtosis = np.where(
    (test_moments.loc[:, "Kurtosis"] < limit_kurtosis[0])
    | (test_moments.loc[:, "Kurtosis"] > limit_kurtosis[1])
)[0]

abn_test_max_abs = np.where(
    (test_moments.loc[:, "MaxAbs"] < limit_max_abs[0])
    | (test_moments.loc[:, "MaxAbs"] > limit_max_abs[1])
)[0]
abn_test_sp_skew = np.where(
    (test_moments.loc[:, "SpSkew"] < limit_sp_skew[0])
    | (test_moments.loc[:, "SpSkew"] > limit_sp_skew[1])
)[0]

allAbnormalities = np.unique(
    np.concatenate(
        (
            abn_test_mean,
            abn_test_variance,
            abn_test_skew,
            abn_test_kurtosis,
            abn_test_max_abs,
            abn_test_sp_skew,
        )
    )
)


# compare sets together. Here I loop over each pair of set and compute how many ID they have in common
test_dict = {
    "Mean": abn_test_mean,
    "Variance": abn_test_variance,
    "Skewness": abn_test_skew,
    "Kurtosis": abn_test_kurtosis,
    "MaxAbs": abn_test_max_abs,
    "SpSkew": abn_test_sp_skew,
}
keysTest = list(test_dict.keys())


print("\nTEST DATASET")
for ii in range(test_dict.__len__()):
    k1 = keysTest[ii]
    print(f"{k1}: {len(test_dict[k1])} anomalies found.")

countAbnormalTest = np.array(
    [np.sum([i in test_dict[k] for k in test_dict]) for i in range(test_set_size)]
)
for i in range(1, 5):
    print(
        f"{len(np.where(countAbnormalTest >= i)[0])} samples are found as anomalous "
        f"by at least {i} selection methods."
    )


In [ ]:
fft = np.abs(np.fft.fft(train_set[1]))[: train_set_timesteps // 2]
plt.plot(fft, label="Normal test sample")
fft = np.abs(np.fft.fft(test_set[0]))[: test_set_timesteps // 2]
plt.plot(fft, label="Abnormal test sample")
plt.legend()


In [ ]:
plt.figure(3, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Fig.4:Venn diagram between recordings considered abnormals by features 'Mean' and 'Variance'.",
    fontsize=20,
)

loc_train_mean = (train_moments.loc[:, "Mean"] < limit_mean[0]) | (
    train_moments.loc[:, "Mean"] > limit_mean[1]
)
loc_train_variance = (train_moments.loc[:, "Variance"] < limit_variance[0]) | (
    train_moments.loc[:, "Variance"] > limit_variance[1]
)
a1 = np.sum(loc_train_mean & loc_train_variance)
a2 = np.sum(loc_train_mean & ~loc_train_variance)
a3 = np.sum(~loc_train_mean & loc_train_variance)

plt.subplot(121)
venn2(subsets=(a2, a3, a1), set_labels=("Mean", "Variance"))
plt.title("Training dataset")


loc_test_mean = (test_moments.loc[:, "Mean"] < limit_mean[0]) | (
    test_moments.loc[:, "Mean"] > limit_mean[1]
)
loc_test_variance = (test_moments.loc[:, "Variance"] < limit_variance[0]) | (
    test_moments.loc[:, "Variance"] > limit_variance[1]
)
a1 = np.sum(loc_test_mean & loc_test_variance)
a2 = np.sum(loc_test_mean & ~loc_test_variance)
a3 = np.sum(~loc_test_mean & loc_test_variance)


plt.subplot(122)
venn2(subsets=(a2, a3, a1), set_labels=("Mean", "Variance"))
plt.title("Test dataset")


In [ ]:
plt.figure(3, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Fig.5:Venn diagram between recordings considered abnormals by features 'Kurtosis' and 'MaxAbs'.",
    fontsize=20,
)

loc_train_kurtosis = (train_moments.loc[:, "Kurtosis"] < limit_kurtosis[0]) | (
    train_moments.loc[:, "Kurtosis"] > limit_kurtosis[1]
)
loc_train_max_abs = (train_moments.loc[:, "MaxAbs"] < limit_max_abs[0]) | (
    train_moments.loc[:, "MaxAbs"] > limit_max_abs[1]
)
a1 = np.sum(loc_train_kurtosis & loc_train_max_abs)
a2 = np.sum(loc_train_kurtosis & ~loc_train_max_abs)
a3 = np.sum(~loc_train_kurtosis & loc_train_max_abs)

plt.subplot(121)
venn2(subsets=(a2, a3, a1), set_labels=("Kurtosis", "MaxAbs"))
plt.title("Training dataset")


loc_test_kurtosis = (test_moments.loc[:, "Kurtosis"] < limit_kurtosis[0]) | (
    test_moments.loc[:, "Kurtosis"] > limit_kurtosis[1]
)
loc_test_max_abs = (test_moments.loc[:, "MaxAbs"] < limit_max_abs[0]) | (
    test_moments.loc[:, "MaxAbs"] > limit_max_abs[1]
)
a1 = np.sum(loc_test_kurtosis & loc_test_max_abs)
a2 = np.sum(loc_test_kurtosis & ~loc_test_max_abs)
a3 = np.sum(~loc_test_kurtosis & loc_test_max_abs)


plt.subplot(122)
venn2(subsets=(a2, a3, a1), set_labels=("Kurtosis", "MaxAbs"))
plt.title("Test dataset")


In [ ]:
plt.figure(3, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Fig.6:Venn diagram between recordings considered abnormals by features 'Kurtosis' and 'SpSkew'.",
    fontsize=20,
)

loc_train_kurtosis = (train_moments.loc[:, "Kurtosis"] < limit_kurtosis[0]) | (
    train_moments.loc[:, "Kurtosis"] > limit_kurtosis[1]
)
loc_train_sp_skew = (train_moments.loc[:, "SpSkew"] < limit_sp_skew[0]) | (
    train_moments.loc[:, "SpSkew"] > limit_sp_skew[1]
)
a1 = np.sum(loc_train_kurtosis & loc_train_sp_skew)
a2 = np.sum(loc_train_kurtosis & ~loc_train_sp_skew)
a3 = np.sum(~loc_train_kurtosis & loc_train_sp_skew)

plt.subplot(121)
venn2(subsets=(a2, a3, a1), set_labels=("Kurtosis", "SpSkew"))
plt.title("Training dataset")


loc_test_kurtosis = (test_moments.loc[:, "Kurtosis"] < limit_kurtosis[0]) | (
    test_moments.loc[:, "Kurtosis"] > limit_kurtosis[1]
)
loc_test_sp_skew = (test_moments.loc[:, "SpSkew"] < limit_sp_skew[0]) | (
    test_moments.loc[:, "SpSkew"] > limit_sp_skew[1]
)
a1 = np.sum(loc_test_kurtosis & loc_test_sp_skew)
a2 = np.sum(loc_test_kurtosis & ~loc_test_sp_skew)
a3 = np.sum(~loc_test_kurtosis & loc_test_sp_skew)


plt.subplot(122)
venn2(subsets=(a2, a3, a1), set_labels=("Kurtosis", "SpSkew"))
plt.title("Test dataset")


In [ ]:
loc_test_kurtosis = (test_moments.loc[:, "Kurtosis"] < limit_kurtosis[0]) | (
    test_moments.loc[:, "Kurtosis"] > limit_kurtosis[1]
)
loc_test_mean = (test_moments.loc[:, "Mean"] < limit_mean[0]) | (
    test_moments.loc[:, "Mean"] > limit_mean[1]
)
loc_test_sp_skew = (test_moments.loc[:, "SpSkew"] < limit_sp_skew[0]) | (
    test_moments.loc[:, "SpSkew"] > limit_sp_skew[1]
)

a1 = np.sum((loc_test_sp_skew & ~loc_test_kurtosis) & ~loc_test_mean)
a2 = np.sum((~loc_test_sp_skew & loc_test_kurtosis) & ~loc_test_mean)
a3 = np.sum((loc_test_sp_skew & loc_test_kurtosis) & ~loc_test_mean)
a4 = np.sum((~loc_test_sp_skew & ~loc_test_kurtosis) & loc_test_mean)
a5 = np.sum((loc_test_sp_skew & ~loc_test_kurtosis) & loc_test_mean)
a6 = np.sum((~loc_test_sp_skew & loc_test_kurtosis) & loc_test_mean)
a7 = np.sum((loc_test_sp_skew & loc_test_kurtosis) & loc_test_mean)

plt.figure(4, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Fig.7:Venn diagram between three features (for the Test dataset).", fontsize=20
)

venn3(subsets=(a1, a2, a3, a4, a5, a6, a7), set_labels=("SpSkew", "Kurtosis", "Mean"))


---

## Part 1: Exercise

⚠️ **This part contains the actual graded exercise 1 questions.** ⚠️

Find your own features for a real acoustic dataset from industrial machines!

Acoustic emission and vibration monitoring play integral roles in structural health 
monitoring (SHM) by providing crucial insights into the condition and integrity of 
structures. Acoustic Emission involves the detection of transient stress waves or 
acoustic signals emitted by materials when they undergo deformation or damage. Vibration 
monitoring with accelerometers is widely used to assess the structural integrity of 
various systems, from bridges and buildings to rotating machinery. Vibration analysis 
can reveal changes in natural frequencies, mode shapes, and damping characteristics, 
which can be indicative of structural damage or degradation. 

### Dataset Description

For this exercise, we consider acoustic emissions from two machines: a **pump** and a 
**valve**. The datasets come as follows:

* **pump**
    * train set containing data from pump in good working conditions
    * test set containing both healthy data and data from pumps with abnormal behaviors
* **valve**
    * train set containing data from valve in good working conditions
    * test set containing both healthy data and data from valves with abnormal behaviors

### Problem Description

Using features introduced in [Part 0](#part-0-demonstration), other features presented 
during lectures and previous exercises, or any other features you deem useful, your goal
is to identify features that can uncover anomalies present in the test dataset but not 
present in the training dataset. Selected features will be different for the pump and 
valve.

### Questions

**Question 1:** Generate plots of raw signals, FFT spectrums or spectrogram from the 
healthy data of both the pump and valve acoustic signals. Discuss the distinctions 
between the signals emitted by these two machines.

Questions 2, 3 and 4 have to be answered separately for both the pump and valve datasets.

**Question 2:** Visualize the raw signals, spectrum, and spectrograms of the test 
dataset for the pump/valve dataset. Are there any signals that appear abnormal to you?

**Question 3:** Compute basic statistical features (mean, variance, skewness, and 
kurtosis) for both the training and test datasets of the pump and valve. Are there any 
abnormal signals that you can detect?

**Question 4:** Find by yourself a feature or a combination of feature that help to 
uncover signals with abnormal behavior. Analyze whether the selected features trigger 
alarms for similar behavior or if some are specific to particular anomalies.

**Question 5:** What are some potential limitations of the method suggested in this 
exercise for anomaly detection? Answer with at least 3 limitations.

**Question 6:** Now that you have developed a set of features and thresholds for a 
valve/pump, imagine applying them to a different valve/pump. Would the discriminative 
power change? If so, how do you propose to mitigate it? Justify and provide concrete 
details.

**Question 7:** Now imagine you're in a scenario where the are no anomalous samples 
available. How would you tackle this problem? Answer with an overview of your proposed 
approach and then provide concrete details regarding the method.


The answers to those questions are expected in a **PDF report**. The full Jupyter notebook 
must also be submitted. A PDF report without any description, analysis, and discussion 
around the plots will not be considered valid.

### Critical Points & Requirements

Here is a unorded non-exhaustive list of some critical points which will be taken into 
account for grading:
- Quality of the plots (legend, axes labels, plot titles, units, ...)
- Discriminative power of the selected features (performance) on the `valve` dataset
- Discriminative power of the selected features (performance) on the `pump` dataset
- Quality of the scientific reasoning
- Structure, clarity, and conciseness of the report
- Quality of the code (useful but concise documentation, clarity of the code, ...)
- ...

⚠️ Your submitted notebook should be able to run on a properly configured environment
(*c.f.* week 1 exercises). If you require additional packages, make sure to add a 
`pip install` command in a code cell (example in the following cell to install `gdown`.)

### 0. Download the Dataset

``gdown`` is a Python library and command-line tool that simplifies the process of downloading files from Google Drive.

In [ ]:
%pip install gdown --upgrade --quiet
import gdown
from scipy.signal import find_peaks



In [ ]:
# Pump dataset
gdown.download(id="1Vz1fhpu5xKJ4RI5maJh_3OweX9BpjBaM", output="./pump.zip")


In [ ]:
# Valve dataset
gdown.download(id="13bzdjL0gEc9hsGHjr0Qo8OMcMF5CrYbS", output="./valve.zip")


### 1. Extract the Datasets

In [23]:
import zipfile
import os

# List all ZIP files in the current directory
zip_files = [file for file in os.listdir() if file.endswith(".zip")]

# Loop through each ZIP file and unzip it in the same directory
for zip_file in zip_files:
    with zipfile.ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall()


### 2. Load the Audio Data

In this code cell, we are using the `librosa` library to load audio files from two 
directories: `/valve` and `/pump`. The goal is to prepare audio data for further 
analysis or processing.


In [24]:
def load_wav_files(directory: str) -> np.ndarray:
    """Load all .WAV audio files in a directory as a 2D Numpy array

    Args:
        directory (str): Path to the directory containing the audio files

    Returns:
        np.ndarray: Loaded audio files as a 2D array
    """
    audio_data: list = []

    # Iterate through files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".wav") or filename.endswith(".WAV"):
            file_path = os.path.join(directory, filename)

            # Load audio file and append it to the list
            audio, sr = librosa.load(file_path, sr=None)
            audio_data.append(audio)

    return np.array(audio_data)


# Paths to the "valve" and "pump" directories
valve_directory = "./valve"
pump_directory = "./pump"

# Load audio data for "valve" and "pump"
valve_train_data = load_wav_files(os.path.join(valve_directory, "train"))
valve_test_data = load_wav_files(os.path.join(valve_directory, "test"))
pump_train_data = load_wav_files(os.path.join(pump_directory, "train"))
pump_test_data = load_wav_files(os.path.join(pump_directory, "test"))


In [ ]:
valve_train_data_size = valve_train_data.shape
valve_test_data_size = valve_test_data.shape
pump_train_data_size = pump_train_data.shape
pump_test_data_size = pump_test_data.shape

print(f"Valve Train Data Shape: {valve_train_data_size}")
print(f"Valve Test Data Shape: {valve_test_data_size}")
print(f"Pump Train Data Shape: {pump_train_data_size}")
print(f"Pump Test Data Shape: {pump_test_data_size}")


⚠️ **You're expected to fill the notebook from here.** ⚠️

### Question 1:


In [26]:
# Define a subfolder in the current project directory
output_dir = 'Figures'

# Create the subfolder if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [27]:
def plot_valve_samples(valve_data, num_samples=3,name = "valve_or_pump", sr=16000):
    """
    Plots the time domain, frequency domain (FFT), and spectrograms for randomly selected samples from the valve dataset.

    Parameters:
    valve_data (numpy array): The dataset containing valve samples, each row is a separate sample.
    num_samples (int): Number of samples to select and plot. Default is 3.
    sr (int): The sample rate for the signals. Default is 16000 Hz.

    Returns:
    None
    """
    # Randomly choose 'num_samples' rows from the dataset
    random_indices = np.random.choice(valve_data.shape[0], num_samples, replace=False)

    # Create subplots: 3 rows (Time Domain, FFT, Spectrogram) and `num_samples` columns
    fig, axes = plt.subplots(3, num_samples, figsize=(18, 18))
    fig.tight_layout(pad=4.0)

    # Loop through each selected sample and plot
    for i, idx in enumerate(random_indices):
        # Get the sample data
        one_valve_sample = valve_data[idx, :]

        # Plot the time domain signal in the first row
        axes[0, i].plot(np.linspace(0, len(one_valve_sample) / sr, len(one_valve_sample)), one_valve_sample, 'b')
        axes[0, i].set_title(f'Time Domain - Sample {idx}')
        axes[0, i].set_xlabel('Time (s)')
        axes[0, i].set_ylabel('Amplitude')

        # Perform FFT
        X = np.fft.fft(one_valve_sample)
        N = len(X)
        n = np.arange(N)
        T = N / sr
        freq = n / T

        # Plot the frequency domain (FFT) in the second row
        axes[1, i].stem(freq, np.abs(X), 'b', markerfmt=" ", basefmt="-b")
        axes[1, i].set_title(f'Freq Domain - Sample {idx}')
        axes[1, i].set_xlabel('Freq (Hz)')
        axes[1, i].set_ylabel('FFT Amplitude')
        axes[1, i].set_xlim(0, sr/2)

        # Perform Spectrogram
        spec = librosa.stft(one_valve_sample, n_fft=1024, hop_length=512, window='hann')
        spec_db = 20 * np.log10(np.abs(spec) + 1e-10)  # Convert to dB scale

        # Plot the Spectrogram in the third row
        vmin = -80
        vmax = 0
        img = librosa.display.specshow(spec_db, sr=sr, hop_length=512, x_axis='time', y_axis='linear', ax=axes[2, i], vmin=vmin, vmax=vmax)
        fig.colorbar(img, ax=axes[2, i], format="%+2.0f dB")  # Add colorbar to the spectrogram

        axes[2, i].set_title(f'Spectrogram - Sample {idx}')
        axes[2, i].set_xlabel('Time (s)')
        axes[2, i].set_ylabel('Freq (Hz)')
    if name != "valve_or_pump":

        # Define the full file path for the image
        file_path = os.path.join(output_dir, f'{name}_healthy_sample.png')

        # Save the plot to the subfolder
        plt.savefig(file_path, dpi=300)

    # Show the plots
    plt.show()


#### Train Valve

In [ ]:

plot_valve_samples(valve_train_data, num_samples=3, name="valve_train")


#### Train Pump

In [ ]:
plot_valve_samples(pump_train_data, num_samples=3, name="pump_train")

### Question 2:

#### Test Valve

In [ ]:

plot_valve_samples(valve_test_data, num_samples=5)


#### Test Pump

In [ ]:
plot_valve_samples(pump_test_data, num_samples=5)

#### Ploting Specific cases

In [32]:
# Valve
# Sample : 31, 98
valve_indicies = [31, 98]

# Pump
# Sample : 103, 131, 151, 137
pump_indicies = [103, 131, 151, 137]

In [ ]:

sr = 16000
# Create subplots: 3 rows (Time Domain, FFT, Spectrogram) and `num_samples` columns
fig, axes = plt.subplots(3, len(valve_indicies), figsize=(18, 18))
fig.tight_layout(pad=4.0)

# Loop through each selected sample and plot
for i, idx in enumerate(valve_indicies):
    # Get the sample data
    one_valve_sample = valve_test_data[idx, :]

    # Plot the time domain signal in the first row
    axes[0, i].plot(np.linspace(0, len(one_valve_sample) / sr, len(one_valve_sample)), one_valve_sample, 'b')
    axes[0, i].set_title(f'Time Domain - Sample {idx}')
    axes[0, i].set_xlabel('Time (s)')
    axes[0, i].set_ylabel('Amplitude')

    # Perform FFT
    X = np.fft.fft(one_valve_sample)
    N = len(X)
    n = np.arange(N)
    T = N / sr
    freq = n / T

    # Plot the frequency domain (FFT) in the second row
    axes[1, i].stem(freq, np.abs(X), 'b', markerfmt=" ", basefmt="-b")
    axes[1, i].set_title(f'Freq Domain - Sample {idx}')
    axes[1, i].set_xlabel('Freq (Hz)')
    axes[1, i].set_ylabel('FFT Amplitude')
    axes[1, i].set_xlim(0, sr/2)

    # Perform Spectrogram
    spec = librosa.stft(one_valve_sample, n_fft=1024, hop_length=512, window='hann')
    spec_db = 20 * np.log10(np.abs(spec) + 1e-10)  # Convert to dB scale

    # Plot the Spectrogram in the third row
    vmin = -80
    vmax = 0
    img = librosa.display.specshow(spec_db, sr=sr, hop_length=512, x_axis='time', y_axis='linear', ax=axes[2, i], vmin=vmin, vmax=vmax)
    fig.colorbar(img, ax=axes[2, i], format="%+2.0f dB")  # Add colorbar to the spectrogram

    axes[2, i].set_title(f'Spectrogram - Sample {idx}')
    axes[2, i].set_xlabel('Time (s)')
    axes[2, i].set_ylabel('Freq (Hz)')

    # Show the plots

# Define the full file path for the image
file_path = os.path.join(output_dir, 'valve_anomalies_sample.png')

# Save the plot to the subfolder
plt.savefig(file_path, dpi=300)
plt.show()


In [ ]:

sr = 16000
# Create subplots: 3 rows (Time Domain, FFT, Spectrogram) and `num_samples` columns
fig, axes = plt.subplots(3, len(pump_indicies), figsize=(18, 18))
fig.tight_layout(pad=4.0)

# Loop through each selected sample and plot
for i, idx in enumerate(pump_indicies):
    # Get the sample data
    one_valve_sample = pump_test_data[idx, :]

    # Plot the time domain signal in the first row
    axes[0, i].plot(np.linspace(0, len(one_valve_sample) / sr, len(one_valve_sample)), one_valve_sample, 'b')
    axes[0, i].set_title(f'Time Domain - Sample {idx}')
    axes[0, i].set_xlabel('Time (s)')
    axes[0, i].set_ylabel('Amplitude')

    # Perform FFT
    X = np.fft.fft(one_valve_sample)
    N = len(X)
    n = np.arange(N)
    T = N / sr
    freq = n / T

    # Plot the frequency domain (FFT) in the second row
    axes[1, i].stem(freq, np.abs(X), 'b', markerfmt=" ", basefmt="-b")
    axes[1, i].set_title(f'Freq Domain - Sample {idx}')
    axes[1, i].set_xlabel('Freq (Hz)')
    axes[1, i].set_ylabel('FFT Amplitude')
    axes[1, i].set_xlim(0, sr/2)

    # Perform Spectrogram
    spec = librosa.stft(one_valve_sample, n_fft=1024, hop_length=512, window='hann')
    spec_db = 20 * np.log10(np.abs(spec) + 1e-10)  # Convert to dB scale

    # Plot the Spectrogram in the third row
    vmin = -80
    vmax = 0
    img = librosa.display.specshow(spec_db, sr=sr, hop_length=512, x_axis='time', y_axis='linear', ax=axes[2, i], vmin=vmin, vmax=vmax)
    fig.colorbar(img, ax=axes[2, i], format="%+2.0f dB")  # Add colorbar to the spectrogram

    axes[2, i].set_title(f'Spectrogram - Sample {idx}')
    axes[2, i].set_xlabel('Time (s)')
    axes[2, i].set_ylabel('Freq (Hz)')

# Define the full file path for the image
file_path = os.path.join(output_dir, 'pump_anomalies_sample.png')

# Save the plot to the subfolder
plt.savefig(file_path, dpi=300)
plt.show()
# Show the plots
plt.show()


### Question 3:

In [35]:
# Compute moments 1 to 4 on the valve training set and assign them in a Pandas dataframe
valve_train_moments = pd.DataFrame(
    {
        "Mean": np.mean(valve_train_data, axis=1),
        "Variance": scist.moment(valve_train_data, moment=2, axis=1, nan_policy="propagate"),
        "Skewness": scist.skew(valve_train_data, axis=1, bias=True, nan_policy="propagate"),
        "Kurtosis": scist.kurtosis(
            valve_train_data, axis=1, fisher=False, bias=True, nan_policy="propagate"
        ),
        "type": "train",
    }
)

# Compute moments 1 to 4 on the valve testing set and assign them in a Pandas dataframe
valve_test_moments = pd.DataFrame(
    {
        "Mean": np.mean(valve_test_data, axis=1),
        "Variance": scist.moment(valve_test_data, moment=2, axis=1, nan_policy="propagate"),
        "Skewness": scist.skew(valve_test_data, axis=1, bias=True, nan_policy="propagate"),
        "Kurtosis": scist.kurtosis(
            valve_test_data, axis=1, fisher=False, bias=True, nan_policy="propagate"
        ),
        "type": "test",
    }
)

# Compute moments 1 to 4 on the pump training set and assign them in a Pandas dataframe
pump_train_moments = pd.DataFrame(
    {
        "Mean": np.mean(pump_train_data, axis=1),
        "Variance": scist.moment(pump_train_data, moment=2, axis=1, nan_policy="propagate"),
        "Skewness": scist.skew(pump_train_data, axis=1, bias=True, nan_policy="propagate"),
        "Kurtosis": scist.kurtosis(
            pump_train_data, axis=1, fisher=False, bias=True, nan_policy="propagate"
        ),
        "type": "train",
    }
)

# Compute moments 1 to 4 on the pump testing set and assign them in a Pandas dataframe
pump_test_moments = pd.DataFrame(
    {
        "Mean": np.mean(pump_test_data, axis=1),
        "Variance": scist.moment(pump_test_data, moment=2, axis=1, nan_policy="propagate"),
        "Skewness": scist.skew(pump_test_data, axis=1, bias=True, nan_policy="propagate"),
        "Kurtosis": scist.kurtosis(
            pump_test_data, axis=1, fisher=False, bias=True, nan_policy="propagate"
        ),
        "type": "test"
    }
)



#### Valve ####

In [ ]:
quantiles = [0.5, 99.5]
limit_mean = np.percentile(valve_train_moments.loc[:, "Mean"], quantiles)
limit_variance = np.percentile(valve_train_moments.loc[:, "Variance"], quantiles)
lim_skew = np.percentile(valve_train_moments.loc[:, "Skewness"], quantiles)
limit_kurtosis = np.percentile(valve_train_moments.loc[:, "Kurtosis"], quantiles)
valve_train_percentage = np.linspace(0, 100, valve_train_data_size[0])  # From 0% to 100%


#Plotting the percentiles of mean, variance, skew and kurtosis for the valve training set
plt.figure(1, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Fig.1: Percentiles of the 4 first centered Moment for the training set for the valve.",
    fontsize=20,
)
plt.subplot(2, 2, 1)
plt.plot(
    np.sort(valve_train_moments.loc[:, "Mean"]), valve_train_percentage, marker=".", linewidth=0
)
plt.plot([limit_mean[0], limit_mean[0]], plt.ylim(), color="k")
plt.plot([limit_mean[1], limit_mean[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Mean-percentile")
plt.ylabel("Percentile")
plt.subplot(2, 2, 2)
plt.plot(
    np.sort(valve_train_moments.loc[:, "Variance"]), valve_train_percentage, marker=".", linewidth=0
)
plt.plot([limit_variance[0], limit_variance[0]], plt.ylim(), color="k")
plt.plot([limit_variance[1], limit_variance[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Variance-percentile")
plt.ylabel("Percentile")
plt.subplot(2, 2, 3)
plt.plot(
    np.sort(valve_train_moments.loc[:, "Skewness"]), valve_train_percentage, marker=".", linewidth=0
)
plt.plot([lim_skew[0], lim_skew[0]], plt.ylim(), color="k")
plt.plot([lim_skew[1], lim_skew[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Skewness-percentile")
plt.ylabel("Percentile")
plt.subplot(2, 2, 4)
plt.plot(
    np.sort(valve_train_moments.loc[:, "Kurtosis"]), valve_train_percentage, marker=".", linewidth=0
)
plt.plot([limit_kurtosis[0], limit_kurtosis[0]], plt.ylim(), color="k")
plt.plot([limit_kurtosis[1], limit_kurtosis[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Kurtosis-percentile")
plt.ylabel("Percentile")

# Define the full file path for the image
file_path = os.path.join(output_dir, 'percentiles_moments_valve_train.png')

# Save the plot to the subfolder
plt.savefig(file_path, dpi=300)
plt.show()


In [ ]:
quantiles = [0.2, 99.8]
limit_mean = np.percentile(valve_train_moments.loc[:, "Mean"], quantiles)
limit_variance = np.percentile(valve_train_moments.loc[:, "Variance"], quantiles)
lim_skew = np.percentile(valve_train_moments.loc[:, "Skewness"], quantiles)
limit_kurtosis = np.percentile(valve_train_moments.loc[:, "Kurtosis"], quantiles)

valve_test_percentage = np.linspace(0, 100, valve_test_data_size[0])  # from 0% to 100% with step 100/nTrain

plt.figure(2, figsize=(15, 10))
plt.clf()
plt.suptitle(
    "Fig.3: Percentiles of the 4 first centered moments for the valve \n" 
    + "for both training and test sets with proposed thresholds.",
    fontsize=20,
)
plt.subplot(3, 2, 1)
plt.plot(np.sort(valve_train_moments.loc[:, "Mean"]), valve_train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(valve_test_moments.loc[:, "Mean"]), valve_test_percentage, marker=".", linewidth=0)
plt.plot([limit_mean[0], limit_mean[0]], plt.ylim(), color="k")
plt.plot([limit_mean[1], limit_mean[1]], plt.ylim(), color="k")
plt.xlabel("Mean")
plt.ylabel("Percentile")

plt.subplot(3, 2, 2)
plt.plot(np.sort(valve_train_moments.loc[:, "Variance"]), valve_train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(valve_test_moments.loc[:, "Variance"]), valve_test_percentage, marker=".", linewidth=0)
plt.plot([limit_variance[0], limit_variance[0]], plt.ylim(), color="k")
plt.plot([limit_variance[1], limit_variance[1]], plt.ylim(), color="k")
plt.xlabel("Variance")
plt.ylabel("Percentile")

plt.subplot(3, 2, 3)
plt.plot(np.sort(valve_train_moments.loc[:, "Skewness"]), valve_train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(valve_test_moments.loc[:, "Skewness"]), valve_test_percentage, marker=".", linewidth=0)
plt.plot([lim_skew[0], lim_skew[0]], plt.ylim(), color="k")
plt.plot([lim_skew[1], lim_skew[1]], plt.ylim(), color="k")
plt.xlabel("Skewness")
plt.ylabel("Percentile")
# plt.xlim([-50,50])

plt.subplot(3, 2, 4)
plt.plot(np.sort(valve_train_moments.loc[:, "Kurtosis"]), valve_train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(valve_test_moments.loc[:, "Kurtosis"]), valve_test_percentage, marker=".", linewidth=0)
plt.plot([limit_kurtosis[0], limit_kurtosis[0]], plt.ylim(), color="k")
plt.plot([limit_kurtosis[1], limit_kurtosis[1]], plt.ylim(), color="k")
plt.xlabel("Kurtosis")
plt.ylabel("Percentile")
# plt.xlim([-50,5000])
plt.figlegend(
    ["Train", "Test", "Threshold"], loc="lower center", ncol=5, labelspacing=0.0
)

# Define the full file path for the image
file_path = os.path.join(output_dir, 'percentiles_moments_valve_train_and_test.png')

# Save the plot to the subfolder
plt.savefig(file_path, dpi=300)
plt.show()


In [ ]:
# select points either below 0.5 percentile or above 99.5 percentile of each moments and min-max limits
abn_valve_train_mean = np.where(
    (valve_train_moments.loc[:, "Mean"] < limit_mean[0])
    | (valve_train_moments.loc[:, "Mean"] > limit_mean[1])
)[0]
abn_valve_train_variance = np.where(
    (valve_train_moments.loc[:, "Variance"] < limit_variance[0])
    | (valve_train_moments.loc[:, "Variance"] > limit_variance[1])
)[0]
abn_valve_train_skew = np.where(
    (valve_train_moments.loc[:, "Skewness"] < lim_skew[0])
    | (valve_train_moments.loc[:, "Skewness"] > lim_skew[1])
)[0]
abn_valve_train_kurtosis = np.where(
    (valve_train_moments.loc[:, "Kurtosis"] < limit_kurtosis[0])
    | (valve_train_moments.loc[:, "Kurtosis"] > limit_kurtosis[1])
)[0]

all_abnormalities_valve_train = np.unique(
    np.concatenate(
        (
            abn_valve_train_mean,
            abn_valve_train_variance,
            abn_valve_train_skew,
            abn_valve_train_kurtosis
        )
    )
)

# Compare sets together. Loop over each pair of sets and compute how many IDs they have in common
valve_train_dict = {
    "Mean": abn_valve_train_mean,
    "Variance": abn_valve_train_variance,
    "Skewness": abn_valve_train_skew,
    "Kurtsosis": abn_valve_train_kurtosis
}
valve_train_dict_keys = list(valve_train_dict.keys())

print("VALVE TRAINING DATASET")
for i in range(valve_train_dict.__len__()):
    k1 = valve_train_dict_keys[i]
    print("{}: {} anomalies found.".format(k1, valve_train_dict[k1].__len__()))

anomalies_count_valve_train = np.array(
    [np.sum([i in valve_train_dict[k] for k in valve_train_dict]) for i in range(valve_train_data_size[0])]
)
for i in range(1, 5):
    print(
        "{} samples are found as anomalous by at least {} selection methods.".format(
            len(np.where(anomalies_count_valve_train >= i)[0]), i
        )
    )

######################################################################################################

# select points either below 0.2 percentile or above 99.8 percentile of each moments and min-max limits
abn_valve_test_mean = np.where(
    (valve_test_moments.loc[:, "Mean"] < limit_mean[0])
    | (valve_test_moments.loc[:, "Mean"] > limit_mean[1])
)[0]
abn_valve_test_variance = np.where(
    (valve_test_moments.loc[:, "Variance"] < limit_variance[0])
    | (valve_test_moments.loc[:, "Variance"] > limit_variance[1])
)[0]
abn_valve_test_skew = np.where(
    (valve_test_moments.loc[:, "Skewness"] < lim_skew[0])
    | (valve_test_moments.loc[:, "Skewness"] > lim_skew[1])
)[0]
abn_valve_test_kurtosis = np.where(
    (valve_test_moments.loc[:, "Kurtosis"] < limit_kurtosis[0])
    | (valve_test_moments.loc[:, "Kurtosis"] > limit_kurtosis[1])
)[0]

allAbnormalities = np.unique(
    np.concatenate(
        (
            abn_valve_test_mean,
            abn_valve_test_variance,
            abn_valve_test_skew,
            abn_valve_test_kurtosis
    )
)
)


# compare sets together. Here I loop over each pair of set and compute how many ID they have in common
valve_test_dict = {
    "Mean": abn_valve_test_mean,
    "Variance": abn_valve_test_variance,
    "Skewness": abn_valve_test_skew,
    "Kurtosis": abn_valve_test_kurtosis
}
keysTest = list(valve_test_dict.keys())


print("\nVALVE TEST DATASET")
for ii in range(valve_test_dict.__len__()):
    k1 = keysTest[ii]
    print(f"{k1}: {len(valve_test_dict[k1])} anomalies found.")

countAbnormalTest = np.array(
    [np.sum([i in valve_test_dict[k] for k in valve_test_dict]) for i in range(valve_test_data_size[0])]
)
for i in range(1, 5):
    print(
        f"{len(np.where(countAbnormalTest >= i)[0])} samples are found as anomalous "
        f"by at least {i} selection methods."
    )

#### Pump ####

In [ ]:
quantiles = [0.5, 99.5]
limit_mean = np.percentile(pump_train_moments.loc[:, "Mean"], quantiles)
limit_variance = np.percentile(pump_train_moments.loc[:, "Variance"], quantiles)
lim_skew = np.percentile(pump_train_moments.loc[:, "Skewness"], quantiles)
limit_kurtosis = np.percentile(pump_train_moments.loc[:, "Kurtosis"], quantiles)
pump_train_percentage = np.linspace(0, 100, pump_train_data_size[0])  # From 0% to 100%


#Plotting the percentiles of mean, variance, skew and kurtosis for the valve training set
plt.figure(1, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Fig.1: Percentiles of the 4 first centered Moment for the training set for the pump.",
    fontsize=20,
)
plt.subplot(2, 2, 1)
plt.plot(
    np.sort(pump_train_moments.loc[:, "Mean"]), pump_train_percentage, marker=".", linewidth=0
)
plt.plot([limit_mean[0], limit_mean[0]], plt.ylim(), color="k")
plt.plot([limit_mean[1], limit_mean[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Mean-percentile")
plt.ylabel("Percentile")
plt.subplot(2, 2, 2)
plt.plot(
    np.sort(pump_train_moments.loc[:, "Variance"]), pump_train_percentage, marker=".", linewidth=0
)
plt.plot([limit_variance[0], limit_variance[0]], plt.ylim(), color="k")
plt.plot([limit_variance[1], limit_variance[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Variance-percentile")
plt.ylabel("Percentile")
plt.subplot(2, 2, 3)
plt.plot(
    np.sort(pump_train_moments.loc[:, "Skewness"]), pump_train_percentage, marker=".", linewidth=0
)
plt.plot([lim_skew[0], lim_skew[0]], plt.ylim(), color="k")
plt.plot([lim_skew[1], lim_skew[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Skewness-percentile")
plt.ylabel("Percentile")
plt.subplot(2, 2, 4)
plt.plot(
    np.sort(pump_train_moments.loc[:, "Kurtosis"]), pump_train_percentage, marker=".", linewidth=0
)
plt.plot([limit_kurtosis[0], limit_kurtosis[0]], plt.ylim(), color="k")
plt.plot([limit_kurtosis[1], limit_kurtosis[1]], plt.ylim(), color="k")
plt.xlabel("Value of the Kurtosis-percentile")
plt.ylabel("Percentile")

# Define the full file path for the image
file_path = os.path.join(output_dir, 'percentiles_moments_pump_train.png')

# Save the plot to the subfolder
plt.savefig(file_path, dpi=300)
plt.show()


In [ ]:
quantiles = [0.2, 99.8]
limit_mean = np.percentile(pump_train_moments.loc[:, "Mean"], quantiles)
limit_variance = np.percentile(pump_train_moments.loc[:, "Variance"], quantiles)
lim_skew = np.percentile(pump_train_moments.loc[:, "Skewness"], quantiles)
limit_kurtosis = np.percentile(pump_train_moments.loc[:, "Kurtosis"], quantiles)

pump_test_percentage = np.linspace(0, 100, pump_test_data_size[0])  # from 0% to 100% with step 100/nTrain

plt.figure(2, figsize=(15, 10))
plt.clf()
plt.suptitle(
    "Fig.3: Percentiles of the 4 first centered moments for the pump \n"
    + "for both training and test sets with proposed thresholds.",
    fontsize=20,
)
plt.subplot(3, 2, 1)
plt.plot(np.sort(pump_train_moments.loc[:, "Mean"]), pump_train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(pump_test_moments.loc[:, "Mean"]), pump_test_percentage, marker=".", linewidth=0)
plt.plot([limit_mean[0], limit_mean[0]], plt.ylim(), color="k")
plt.plot([limit_mean[1], limit_mean[1]], plt.ylim(), color="k")
plt.xlabel("Mean")
plt.ylabel("Percentile")

plt.subplot(3, 2, 2)
plt.plot(np.sort(pump_train_moments.loc[:, "Variance"]), pump_train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(pump_test_moments.loc[:, "Variance"]), pump_test_percentage, marker=".", linewidth=0)
plt.plot([limit_variance[0], limit_variance[0]], plt.ylim(), color="k")
plt.plot([limit_variance[1], limit_variance[1]], plt.ylim(), color="k")
plt.xlabel("Variance")
plt.ylabel("Percentile")

plt.subplot(3, 2, 3)
plt.plot(np.sort(pump_train_moments.loc[:, "Skewness"]), pump_train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(pump_test_moments.loc[:, "Skewness"]), pump_test_percentage, marker=".", linewidth=0)
plt.plot([lim_skew[0], lim_skew[0]], plt.ylim(), color="k")
plt.plot([lim_skew[1], lim_skew[1]], plt.ylim(), color="k")
plt.xlabel("Skewness")
plt.ylabel("Percentile")
# plt.xlim([-50,50])

plt.subplot(3, 2, 4)
plt.plot(np.sort(pump_train_moments.loc[:, "Kurtosis"]), pump_train_percentage, marker=".", linewidth=0)
plt.plot(np.sort(pump_test_moments.loc[:, "Kurtosis"]), pump_test_percentage, marker=".", linewidth=0)
plt.plot([limit_kurtosis[0], limit_kurtosis[0]], plt.ylim(), color="k")
plt.plot([limit_kurtosis[1], limit_kurtosis[1]], plt.ylim(), color="k")
plt.xlabel("Kurtosis")
plt.ylabel("Percentile")
# plt.xlim([-50,5000])
plt.figlegend(
    ["Train", "Test", "Threshold"], loc="lower center", ncol=5, labelspacing=0.0
)

# Define the full file path for the image
file_path = os.path.join(output_dir, 'percentiles_moments_pump_train_and_test.png')

# Save the plot to the subfolder
plt.savefig(file_path, dpi=300)
plt.show()


In [ ]:
# select points either below 0.5 percentile or above 99.5 percentile of each moments and min-max limits
abn_pump_train_mean = np.where(
    (pump_train_moments.loc[:, "Mean"] < limit_mean[0])
    | (pump_train_moments.loc[:, "Mean"] > limit_mean[1])
)[0]
abn_pump_train_variance = np.where(
    (pump_train_moments.loc[:, "Variance"] < limit_variance[0])
    | (pump_train_moments.loc[:, "Variance"] > limit_variance[1])
)[0]
abn_pump_train_skew = np.where(
    (pump_train_moments.loc[:, "Skewness"] < lim_skew[0])
    | (pump_train_moments.loc[:, "Skewness"] > lim_skew[1])
)[0]
abn_pump_train_kurtosis = np.where(
    (pump_train_moments.loc[:, "Kurtosis"] < limit_kurtosis[0])
    | (pump_train_moments.loc[:, "Kurtosis"] > limit_kurtosis[1])
)[0]

all_abnormalities_pump_train = np.unique(
    np.concatenate(
        (
            abn_pump_train_mean,
            abn_pump_train_variance,
            abn_pump_train_skew,
            abn_pump_train_kurtosis
        )
    )
)

# Compare sets together. Loop over each pair of sets and compute how many IDs they have in common
pump_train_dict = {
    "Mean": abn_pump_train_mean,
    "Variance": abn_pump_train_variance,
    "Skewness": abn_pump_train_skew,
    "Kurtsosis": abn_pump_train_kurtosis
}
pump_train_dict_keys = list(pump_train_dict.keys())

print("PUMP TRAINING DATASET")
for i in range(pump_train_dict.__len__()):
    k1 = pump_train_dict_keys[i]
    print("{}: {} anomalies found.".format(k1, pump_train_dict[k1].__len__()))

anomalies_count_pump_train = np.array(
    [np.sum([i in pump_train_dict[k] for k in pump_train_dict]) for i in range(pump_train_data_size[0])]
)
for i in range(1, 5):
    print(
        "{} samples are found as anomalous by at least {} selection methods.".format(
            len(np.where(anomalies_count_pump_train >= i)[0]), i
        )
    )

######################################################################################################

# select points either below 0.2 percentile or above 99.8 percentile of each moments and min-max limits
abn_pump_test_mean = np.where(
    (pump_test_moments.loc[:, "Mean"] < limit_mean[0])
    | (pump_test_moments.loc[:, "Mean"] > limit_mean[1])
)[0]
abn_pump_test_variance = np.where(
    (pump_test_moments.loc[:, "Variance"] < limit_variance[0])
    | (pump_test_moments.loc[:, "Variance"] > limit_variance[1])
)[0]
abn_pump_test_skew = np.where(
    (pump_test_moments.loc[:, "Skewness"] < lim_skew[0])
    | (pump_test_moments.loc[:, "Skewness"] > lim_skew[1])
)[0]
abn_pump_test_kurtosis = np.where(
    (pump_test_moments.loc[:, "Kurtosis"] < limit_kurtosis[0])
    | (pump_test_moments.loc[:, "Kurtosis"] > limit_kurtosis[1])
)[0]

allAbnormalities = np.unique(
    np.concatenate(
        (
            abn_pump_test_mean,
            abn_pump_test_variance,
            abn_pump_test_skew,
            abn_pump_test_kurtosis
    )
)
)


# compare sets together. Here I loop over each pair of set and compute how many ID they have in common
pump_test_dict = {
    "Mean": abn_pump_test_mean,
    "Variance": abn_pump_test_variance,
    "Skewness": abn_pump_test_skew,
    "Kurtosis": abn_pump_test_kurtosis
}
keysTest = list(pump_test_dict.keys())


print("PUMP TEST DATASET")
for ii in range(pump_test_dict.__len__()):
    k1 = keysTest[ii]
    print(f"{k1}: {len(pump_test_dict[k1])} anomalies found.")

countAbnormalTest = np.array(
    [np.sum([i in pump_test_dict[k] for k in pump_test_dict]) for i in range(pump_test_data_size[0])]
)
for i in range(1, 5):
    print(
        f"{len(np.where(countAbnormalTest >= i)[0])} samples are found as anomalous "
        f"by at least {i} selection methods."
    )

### Question 4:

In [62]:
def calculate_features_old(signal, sr=16000):
    # Compute Short-Time Fourier Transform (STFT)
    stft = np.abs(librosa.stft(signal, n_fft=1024, hop_length=512, window='hann'))  # Short-Time Fourier Transform (Magnitude)
    
    # Compute Power Spectral Density (PSD)
    psd = stft ** 2
    psd /= np.sum(psd, axis=0, keepdims=True)  # Normalize along time axis
    
    # Frequency bins corresponding to STFT
    freqs = librosa.fft_frequencies(sr=sr, n_fft=1024)

    # Compute Spectral Entropy (Mean across frames)
    spectral_entropy = -np.sum(psd * np.log2(psd + 1e-10), axis=0)
    spectral_entropy_mean = np.mean(spectral_entropy)

    # Compute Spectral Centroid (Mean and Std)
    spectral_centroid = librosa.feature.spectral_centroid(S=stft, sr=sr)
    spectral_centroid_mean = np.mean(spectral_centroid)
    spectral_centroid_std = np.std(spectral_centroid)
    # Compute spectral contrast (No good result)
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr), axis=1)
    spectral_contrast_mean = np.mean(spectral_contrast)
    spectral_contrast_std = np.std(spectral_contrast)


    # Energy Ratios
    low_energy = np.sum(psd[freqs <= 500, :], axis=0)  # Low frequency energy
    high_energy = np.sum(psd[freqs > 3000, :], axis=0)  # High frequency energy
    energy_ratio = np.mean(high_energy / (low_energy + 1e-10))  # Mean ratio across frames
    
    # New energie version tunning the ratio to see if there is better result --> no
    low_energy = np.sum(psd[freqs <= 1000])  # Energy below 1000 Hz
    mid_energy = np.sum(psd[(freqs > 1000) & (freqs <= 3000)])  # Energy between 1000-3000 Hz
    high_energy = np.sum(psd[freqs > 3000])  # Energy above 3000 Hz
    energy_ratio = high_energy / (low_energy + 1e-10)

    # Calculate standard deviation of amplitude values across time for each frequency band
    spec_db = 20 * np.log10(stft + 1e-10)  # Convert to dB scale to prevent log(0)
    std_amplitude = np.mean(np.std(spec_db, axis=1))  # Std across time, then mean over frequency bands

    # Peak prominence in the FFT domain
    fft_values = np.abs(np.fft.rfft(signal)) # real fast fourrier transform 
    fft_freqs = np.fft.rfftfreq(len(signal), 1/sr)
    peak_2000 = max(fft_values[(fft_freqs >= 2000) & (fft_freqs <= 2500)]) if any(fft_values[(fft_freqs >= 2000) & (fft_freqs <= 2500)]) else 0 # if block to handle error

    # Peak analysis using FFT for prominent peaks
    peaks, _ = find_peaks(fft_values, prominence=10)  # Find peaks with prominence threshold
    num_peaks = len(peaks)

    # Autocorrelation feature 
    autocorrelation = librosa.autocorrelate(signal) 
    # "Maximum value: Represents the highest similarity"
    autocorrelation_max = np.max(autocorrelation)
    # Captures the average behavior. 
    autocorrelation_mean = np.mean(autocorrelation)
    autocorrelation_std = np.std(autocorrelation)
    # representing periodicity
    first_peak = np.argmax(autocorrelation[1:]) + 1
    # eflect signal smoothness.
    energy_decay = np.sum(np.abs(autocorrelation))

    # each of the autocorrealtion featues are used in the autocorrelation column and only the most relevant are keeped
    # max, mean are ok
    # std, energy_decay are bad
    # first peak is useless

    # Harmonic to Noise Ratio (HNR) -> Bad
    harmonic, percussive = librosa.effects.hpss(signal)
    hnr = np.mean(librosa.feature.rms(y=harmonic) / (librosa.feature.rms(y=percussive) + 1e-10))

    # Spectral flatterness -> ok
    spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=signal))

    # Zero crossing rate -> ok
    zero_crossings = np.mean(librosa.feature.zero_crossing_rate(y=signal))

    # Spectral Bandwidth -> almost good 
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(S=stft, sr=sr))
    # tuning this feature
    # Spectral Bandwidth Features
    spectral_bandwidth_values = librosa.feature.spectral_bandwidth(S=stft, sr=sr)
    spectral_bandwidth_mean = np.mean(spectral_bandwidth_values)
    spectral_bandwidth_std = np.std(spectral_bandwidth_values)
    spectral_bandwidth_skewness = scist.skew(spectral_bandwidth_values, axis=None)
    spectral_bandwidth_kurtosis = scist.kurtosis(spectral_bandwidth_values, axis=None)
    mean_std_ratio = spectral_bandwidth_mean / (spectral_bandwidth_std + 1e-10)
    max_mean_ratio = np.max(spectral_bandwidth_values) / (spectral_bandwidth_mean + 1e-10)
    spectral_bandwidth_diff = np.diff(spectral_bandwidth_values, axis=1)
    change_rate = np.mean(np.abs(spectral_bandwidth_diff))  # Rate of change





    return {
        "num_peaks": num_peaks,
        "spectral_entropy_mean": spectral_entropy_mean,
        "spectral_centroid_mean": spectral_centroid_mean,
        "spectral_centroid_std": spectral_centroid_std,
        "energy_ratio": energy_ratio,
        "std_spectogram": std_amplitude,
        "peak_2000_prominence": peak_2000,
        "spectral_bandwidth": spectral_bandwidth_mean,
    }


In [109]:
def calculate_features(signal, sr=16000):
    # Compute Short-Time Fourier Transform (STFT)
    stft = np.abs(librosa.stft(signal, n_fft=1024, hop_length=512, window='hann'))

    # Compute Power Spectral Density (PSD) and frequency bins
    psd = stft ** 2
    psd /= np.sum(psd, axis=0, keepdims=True)  # Normalize across time frames
    freqs = librosa.fft_frequencies(sr=sr, n_fft=1024)

    # Compute Spectral Entropy (Mean across frames)
    spectral_entropy = -np.sum(psd * np.log2(psd + 1e-10), axis=0)
    spectral_entropy_mean = np.mean(spectral_entropy)

    # Compute Spectral Centroid (Mean and Std)
    spectral_centroid = librosa.feature.spectral_centroid(S=stft, sr=sr)
    spectral_centroid_mean = np.mean(spectral_centroid)
    spectral_centroid_std = np.std(spectral_centroid)

    # Energy Ratio (High vs. Low Frequency)
    low_energy = np.sum(psd[freqs <= 500, :], axis=0)
    high_energy = np.sum(psd[freqs > 3000, :], axis=0)
    energy_ratio = np.mean(high_energy / (low_energy + 1e-10))

    # Standard Deviation of Amplitude Values (dB)
    spec_db = 20 * np.log10(stft + 1e-10)  # Convert to dB scale
    std_amplitude = np.mean(np.std(spec_db, axis=1))

    # Peak prominence in FFT domain from 2000 Hz to 2500 Hz
    fft_values = np.abs(np.fft.rfft(signal))
    fft_freqs = np.fft.rfftfreq(len(signal), 1/sr)
    peak_2000 = max(fft_values[(fft_freqs >= 2150) & (fft_freqs <= 2350)]) if any(fft_values[(fft_freqs >= 2150) & (fft_freqs <= 2350)]) else 0


    # Count prominent peaks in the FFT
    peaks, _ = find_peaks(fft_values, prominence=10)
    num_peaks = len(peaks)

    # Spectral Bandwidth (captures spread of frequencies)
    spectral_bandwidth_values = librosa.feature.spectral_bandwidth(S=stft, sr=sr)
    spectral_bandwidth_mean = np.mean(spectral_bandwidth_values)
    

    # Return a dictionary with the selected features
    return {
        "num_peaks": num_peaks,
        "spectral_entropy_mean": spectral_entropy_mean,
        "spectral_centroid_mean": spectral_centroid_mean,
        "spectral_centroid_std": spectral_centroid_std,
        "energy_ratio": energy_ratio,
        "std_spectogram": std_amplitude,
        "peak_2000_prominence": peak_2000,
        "spectral_bandwidth": spectral_bandwidth_mean,
    }


In [ ]:
# Create DataFrames for each dataset
def create_feature_dataframe(data):
    # Calculate features for each sample (row)
    features = [calculate_features(data[i]) for i in range(data.shape[0])]
    return pd.DataFrame(features)

# Generate DataFrames
pump_train_df = create_feature_dataframe(pump_train_data)
pump_test_df = create_feature_dataframe(pump_test_data)

print("\nPump Train DataFrame:")
print(pump_train_df.head())


In [ ]:
# Set the quantiles for threshold lines
quantiles = [0.2, 99.8]

# Calculate limits for each feature
limit_num_peaks = np.percentile(pump_train_df['num_peaks'], quantiles)
limit_spectral_entropy = np.percentile(pump_train_df['spectral_entropy_mean'], quantiles)
limit_spectral_centroid_mean = np.percentile(pump_train_df['spectral_centroid_mean'], quantiles)
limit_spectral_centroid_std = np.percentile(pump_train_df['spectral_centroid_std'], quantiles)
limit_energy_ratio = np.percentile(pump_train_df['energy_ratio'], quantiles)
limit_std_spectogram = np.percentile(pump_train_df['std_spectogram'], quantiles)
limit_peak_2000 = np.percentile(pump_train_df['peak_2000_prominence'], quantiles)
limit_spectral_bandwidth = np.percentile(pump_train_df['spectral_bandwidth'], quantiles)


# Get percentage for Pump Test Data
pump_train_percentage = np.linspace(0, 100, pump_train_df.shape[0])
pump_test_percentage = np.linspace(0, 100, pump_test_df.shape[0])

# Set up the matplotlib figure
plt.figure(figsize=(15, 12))
plt.clf()
plt.suptitle(
    "Percentiles of Features for Pump Train and Test Sets with Proposed Thresholds",
    fontsize=20,
)

# Plot for num_peaks
plt.subplot(4, 2, 1)
plt.plot(np.sort(pump_train_df['num_peaks']), pump_train_percentage, marker='.', linewidth=0)
plt.plot(np.sort(pump_test_df['num_peaks']), pump_test_percentage, marker='.', linewidth=0)
plt.plot([limit_num_peaks[0], limit_num_peaks[0]], plt.ylim(), color='k')
plt.plot([limit_num_peaks[1], limit_num_peaks[1]], plt.ylim(), color='k')
plt.xlabel("Number of Peaks")
plt.ylabel("Percentile")
plt.title("Num Peaks")

# Plot for spectral_entropy_mean
plt.subplot(4, 2, 2)
plt.plot(np.sort(pump_train_df['spectral_entropy_mean']), pump_train_percentage, marker='.', linewidth=0)
plt.plot(np.sort(pump_test_df['spectral_entropy_mean']), pump_test_percentage, marker='.', linewidth=0)
plt.plot([limit_spectral_entropy[0], limit_spectral_entropy[0]], plt.ylim(), color='k')
plt.plot([limit_spectral_entropy[1], limit_spectral_entropy[1]], plt.ylim(), color='k')
plt.xlabel("Spectral Entropy Mean")
plt.ylabel("Percentile")
plt.title("Spectral Entropy Mean")

# Plot for spectral_centroid_mean
plt.subplot(4, 2, 3)
plt.plot(np.sort(pump_train_df['spectral_centroid_mean']), pump_train_percentage, marker='.', linewidth=0)
plt.plot(np.sort(pump_test_df['spectral_centroid_mean']), pump_test_percentage, marker='.', linewidth=0)
plt.plot([limit_spectral_centroid_mean[0], limit_spectral_centroid_mean[0]], plt.ylim(), color='k')
plt.plot([limit_spectral_centroid_mean[1], limit_spectral_centroid_mean[1]], plt.ylim(), color='k')
plt.xlabel("Spectral Centroid Mean")
plt.ylabel("Percentile")
plt.title("Spectral Centroid Mean")

# Plot for spectral_centroid_std
plt.subplot(4, 2, 4)
plt.plot(np.sort(pump_train_df['spectral_centroid_std']), pump_train_percentage, marker='.', linewidth=0)
plt.plot(np.sort(pump_test_df['spectral_centroid_std']), pump_test_percentage, marker='.', linewidth=0)
plt.plot([limit_spectral_centroid_std[0], limit_spectral_centroid_std[0]], plt.ylim(), color='k')
plt.plot([limit_spectral_centroid_std[1], limit_spectral_centroid_std[1]], plt.ylim(), color='k')
plt.xlabel("Spectral Centroid Std")
plt.ylabel("Percentile")
plt.title("Spectral Centroid Std")

# Plot for energy_ratio
plt.subplot(4, 2, 5)
plt.plot(np.sort(pump_train_df['energy_ratio']), pump_train_percentage, marker='.', linewidth=0)
plt.plot(np.sort(pump_test_df['energy_ratio']), pump_test_percentage, marker='.', linewidth=0)
plt.plot([limit_energy_ratio[0], limit_energy_ratio[0]], plt.ylim(), color='k')
plt.plot([limit_energy_ratio[1], limit_energy_ratio[1]], plt.ylim(), color='k')
plt.xlabel("Energy Ratio")
plt.ylabel("Percentile")
plt.title("Energy Ratio")

# Plot for std spectogrgam
plt.subplot(4, 2, 6)
plt.plot(np.sort(pump_train_df['std_spectogram']), pump_train_percentage, marker='.', linewidth=0)
plt.plot(np.sort(pump_test_df['std_spectogram']), pump_test_percentage, marker='.', linewidth=0)
plt.plot([limit_std_spectogram[0], limit_std_spectogram[0]], plt.ylim(), color='k')
plt.plot([limit_std_spectogram[1], limit_std_spectogram[1]], plt.ylim(), color='k')
plt.xlabel("Standard Deviation of the spectogram")
plt.ylabel("Percentile")
plt.title("Standard Deviation of the spectogram")

# Plot for peak_2000_prominence
plt.subplot(4, 2, 7)
plt.plot(np.sort(pump_train_df['peak_2000_prominence']), pump_train_percentage, marker='.', linewidth=0)
plt.plot(np.sort(pump_test_df['peak_2000_prominence']), pump_test_percentage, marker='.', linewidth=0)
plt.plot([limit_peak_2000[0], limit_peak_2000[0]], plt.ylim(), color='k')
plt.plot([limit_peak_2000[1], limit_peak_2000[1]], plt.ylim(), color='k')
plt.xlabel("Peak at 2000 Hz Prominence")
plt.ylabel("Percentile")
plt.title("Peak 2000 Hz Prominence")

# Plot for Spectral Bandwidth
plt.subplot(4, 2, 8)
plt.plot(np.sort(pump_train_df['spectral_bandwidth']), pump_train_percentage, marker='.', linewidth=0)
plt.plot(np.sort(pump_test_df['spectral_bandwidth']), pump_test_percentage, marker='.', linewidth=0)
plt.plot([limit_spectral_bandwidth[0], limit_spectral_bandwidth[0]], plt.ylim(), color='k')
plt.plot([limit_spectral_bandwidth[1], limit_spectral_bandwidth[1]], plt.ylim(), color='k')
plt.xlabel("Spectral Bandwidth")
plt.ylabel("Percentile")
plt.title("Spectral Bandwidth")

# Legend
plt.figlegend(["Pump Train", "Pump Test", "Threshold"], loc="lower center", ncol=3, labelspacing=0.0)

# Show the plot
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Adjust layout to fit the title
plt.show()


In [ ]:


# Detect anomalies in the training dataset
abn_num_peaks = np.where(
    (pump_train_df['num_peaks'] < limit_num_peaks[0]) |
    (pump_train_df['num_peaks'] > limit_num_peaks[1])
)[0]

abn_spectral_entropy = np.where(
    (pump_train_df['spectral_entropy_mean'] < limit_spectral_entropy[0]) |
    (pump_train_df['spectral_entropy_mean'] > limit_spectral_entropy[1])
)[0]

abn_spectral_centroid_mean = np.where(
    (pump_train_df['spectral_centroid_mean'] < limit_spectral_centroid_mean[0]) |
    (pump_train_df['spectral_centroid_mean'] > limit_spectral_centroid_mean[1])
)[0]

abn_spectral_centroid_std = np.where(
    (pump_train_df['spectral_centroid_std'] < limit_spectral_centroid_std[0]) |
    (pump_train_df['spectral_centroid_std'] > limit_spectral_centroid_std[1])
)[0]

abn_energy_ratio = np.where(
    (pump_train_df['energy_ratio'] < limit_energy_ratio[0]) |
    (pump_train_df['energy_ratio'] > limit_energy_ratio[1])
)[0]

abn_std_spectrogram = np.where(
    (pump_train_df['std_spectogram'] < limit_std_spectogram[0]) |
    (pump_train_df['std_spectogram'] > limit_std_spectogram[1])
)[0]

abn_peak_2000 = np.where(
    (pump_train_df['peak_2000_prominence'] < limit_peak_2000[0]) |
    (pump_train_df['peak_2000_prominence'] > limit_peak_2000[1])
)[0]

abn_spectral_bandwidth = np.where(
    (pump_train_df['spectral_bandwidth'] < limit_spectral_bandwidth[0]) |
    (pump_train_df['spectral_bandwidth'] > limit_spectral_bandwidth[1])
)[0]

# Collect all abnormal indices
all_abnormalities_train = np.unique(np.concatenate((
    abn_num_peaks,
    abn_spectral_entropy,
    abn_spectral_centroid_mean,
    abn_spectral_centroid_std,
    abn_energy_ratio,
    abn_std_spectrogram,
    abn_peak_2000,
    abn_spectral_bandwidth
)))

# Compare sets together
pump_train_dict = {
    "Num Peaks": abn_num_peaks,
    "Spectral Entropy": abn_spectral_entropy,
    "Spectral Centroid Mean": abn_spectral_centroid_mean,
    "Spectral Centroid Std": abn_spectral_centroid_std,
    "Energy Ratio": abn_energy_ratio,
    "Std Spectrogram": abn_std_spectrogram,
    "Peak 2000 Hz Prominence": abn_peak_2000,
    "Spectral Bandwidth": abn_spectral_bandwidth
}

# Print anomalies found
print("PUMP TRAINING DATASET")
for feature_name, anomalies in pump_train_dict.items():
    print(f"{feature_name}: {len(anomalies)} anomalies found.")

# Counting how many methods flagged each sample
anomalies_count_train = np.array(
    [np.sum([i in pump_train_dict[k] for k in pump_train_dict]) for i in range(pump_train_df.shape[0])]
)

# Print counts of samples flagged as anomalous by at least i selection methods
for i in range(1, 8):
    print(f"{len(np.where(anomalies_count_train >= i)[0])} samples are found as anomalous by at least {i} selection methods.")


In [ ]:
# Detect anomalies in the test dataset
abn_num_peaks_test = np.where(
    (pump_test_df['num_peaks'] < limit_num_peaks[0]) |
    (pump_test_df['num_peaks'] > limit_num_peaks[1])
)[0]

abn_spectral_entropy_test = np.where(
    (pump_test_df['spectral_entropy_mean'] < limit_spectral_entropy[0]) |
    (pump_test_df['spectral_entropy_mean'] > limit_spectral_entropy[1])
)[0]

abn_spectral_centroid_mean_test = np.where(
    (pump_test_df['spectral_centroid_mean'] < limit_spectral_centroid_mean[0]) |
    (pump_test_df['spectral_centroid_mean'] > limit_spectral_centroid_mean[1])
)[0]

abn_spectral_centroid_std_test = np.where(
    (pump_test_df['spectral_centroid_std'] < limit_spectral_centroid_std[0]) |
    (pump_test_df['spectral_centroid_std'] > limit_spectral_centroid_std[1])
)[0]

abn_energy_ratio_test = np.where(
    (pump_test_df['energy_ratio'] < limit_energy_ratio[0]) |
    (pump_test_df['energy_ratio'] > limit_energy_ratio[1])
)[0]

abn_std_spectrogram_test = np.where(
    (pump_test_df['std_spectogram'] < limit_std_spectogram[0]) |
    (pump_test_df['std_spectogram'] > limit_std_spectogram[1])
)[0]

abn_peak_2000_test = np.where(
    (pump_test_df['peak_2000_prominence'] < limit_peak_2000[0]) |
    (pump_test_df['peak_2000_prominence'] > limit_peak_2000[1])
)[0]

abn_spectral_bandwidth = np.where(
    (pump_test_df['spectral_bandwidth'] < limit_spectral_bandwidth[0]) |
    (pump_test_df['spectral_bandwidth'] > limit_spectral_bandwidth[1])
)[0]

# Collect all abnormal indices
all_abnormalities_test = np.unique(np.concatenate((
    abn_num_peaks_test,
    abn_spectral_entropy_test,
    abn_spectral_centroid_mean_test,
    abn_spectral_centroid_std_test,
    abn_energy_ratio_test,
    abn_std_spectrogram_test,
    abn_peak_2000_test,
    abn_spectral_bandwidth
)))

# Compare sets together
pump_test_dict = {
    "Num Peaks": abn_num_peaks_test,
    "Spectral Entropy": abn_spectral_entropy_test,
    "Spectral Centroid Mean": abn_spectral_centroid_mean_test,
    "Spectral Centroid Std": abn_spectral_centroid_std_test,
    "Energy Ratio": abn_energy_ratio_test,
    "Std Spectrogram": abn_std_spectrogram_test,
    "Peak 2000 Hz Prominence": abn_peak_2000_test,
    "Spectral Bandwidth": abn_spectral_bandwidth
}

# Print anomalies found
print("PUMP TEST DATASET")
for feature_name, anomalies in pump_test_dict.items():
    print(f"{feature_name}: {len(anomalies)} anomalies found.")

# Counting how many methods flagged each sample
anomalies_count_test = np.array(
    [np.sum([i in pump_test_dict[k] for k in pump_test_dict]) for i in range(pump_test_df.shape[0])]
)

# Print counts of samples flagged as anomalous by at least i selection methods
for i in range(1, 8):
    print(f"{len(np.where(anomalies_count_test >= i)[0])} samples are found as anomalous by at least {i} selection methods.")

In [ ]:
quantiles = [0.2, 99.8]
limit_mean = np.percentile(valve_train_moments.loc[:, "Mean"], quantiles)
limit_variance = np.percentile(valve_train_moments.loc[:, "Variance"], quantiles)
lim_skew = np.percentile(valve_train_moments.loc[:, "Skewness"], quantiles)
limit_kurtosis = np.percentile(valve_train_moments.loc[:, "Kurtosis"], quantiles)
plt.figure(3, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Venn diagram between recordings considered abnormals by features 'Mean' and 'Variance'.",
    fontsize=20,
)

loc_train_mean = (valve_train_moments.loc[:, "Mean"] < limit_mean[0]) | (
    valve_train_moments.loc[:, "Mean"] > limit_mean[1]
)
loc_train_variance = (valve_train_moments.loc[:, "Variance"] < limit_variance[0]) | (
    valve_train_moments.loc[:, "Variance"] > limit_variance[1]
)
a1 = np.sum(loc_train_mean & loc_train_variance)
a2 = np.sum(loc_train_mean & ~loc_train_variance)
a3 = np.sum(~loc_train_mean & loc_train_variance)

plt.subplot(121)
venn2(subsets=(a2, a3, a1), set_labels=("Mean", "Variance"))
plt.title("Training dataset")


loc_test_mean = (valve_test_moments.loc[:, "Mean"] < limit_mean[0]) | (
    valve_test_moments.loc[:, "Mean"] > limit_mean[1]
)
loc_test_variance = (valve_test_moments.loc[:, "Variance"] < limit_variance[0]) | (
    valve_test_moments.loc[:, "Variance"] > limit_variance[1]
)
a1 = np.sum(loc_test_mean & loc_test_variance)
a2 = np.sum(loc_test_mean & ~loc_test_variance)
a3 = np.sum(~loc_test_mean & loc_test_variance)


plt.subplot(122)
venn2(subsets=(a2, a3, a1), set_labels=("Mean", "Variance"))
plt.title("Test dataset")


In [ ]:
quantiles = [0.2, 99.8]
limit_mean = np.percentile(valve_train_moments.loc[:, "Mean"], quantiles)
limit_variance = np.percentile(valve_train_moments.loc[:, "Variance"], quantiles)
lim_skew = np.percentile(valve_train_moments.loc[:, "Skewness"], quantiles)
limit_kurtosis = np.percentile(valve_train_moments.loc[:, "Kurtosis"], quantiles)
plt.figure(3, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Venn diagram between recordings considered abnormals by features 'kurtosis' and 'Variance'.",
    fontsize=20,
)

loc_train_mean = (valve_train_moments.loc[:, "Kurtosis"] < limit_kurtosis[0]) | (
    valve_train_moments.loc[:, "Kurtosis"] > limit_kurtosis[1]
)
loc_train_variance = (valve_train_moments.loc[:, "Variance"] < limit_variance[0]) | (
    valve_train_moments.loc[:, "Variance"] > limit_variance[1]
)
a1 = np.sum(loc_train_mean & loc_train_variance)
a2 = np.sum(loc_train_mean & ~loc_train_variance)
a3 = np.sum(~loc_train_mean & loc_train_variance)

plt.subplot(121)
venn2(subsets=(a2, a3, a1), set_labels=("Kurtosis", "Variance"))
plt.title("Training dataset")


loc_test_mean = (valve_test_moments.loc[:, "Kurtosis"] < limit_kurtosis[0]) | (
    valve_test_moments.loc[:, "Kurtosis"] > limit_kurtosis[1]
)
loc_test_variance = (valve_test_moments.loc[:, "Variance"] < limit_variance[0]) | (
    valve_test_moments.loc[:, "Variance"] > limit_variance[1]
)
a1 = np.sum(loc_test_mean & loc_test_variance)
a2 = np.sum(loc_test_mean & ~loc_test_variance)
a3 = np.sum(~loc_test_mean & loc_test_variance)


plt.subplot(122)
venn2(subsets=(a2, a3, a1), set_labels=("kurtosis", "Variance"))
plt.title("Test dataset")


In [ ]:
# Set the quantiles for threshold lines
quantiles = [0.2, 99.8]

# Calculate limits for each feature
limit_num_peaks = np.percentile(pump_train_df['num_peaks'], quantiles)
limit_spectral_entropy = np.percentile(pump_train_df['spectral_entropy_mean'], quantiles)
limit_spectral_centroid_mean = np.percentile(pump_train_df['spectral_centroid_mean'], quantiles)
limit_spectral_centroid_std = np.percentile(pump_train_df['spectral_centroid_std'], quantiles)
limit_energy_ratio = np.percentile(pump_train_df['energy_ratio'], quantiles)
limit_std_spectogram = np.percentile(pump_train_df['std_spectogram'], quantiles)
limit_peak_2000 = np.percentile(pump_train_df['peak_2000_prominence'], quantiles)
limit_spectral_bandwidth = np.percentile(pump_train_df['spectral_bandwidth'], quantiles)

plt.figure(3, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Venn diagram between recordings considered abnormals by features 'peak_2000_prominence' and 'Variance'.",
    fontsize=20,
)

loc_train_mean = (pump_train_df.loc[:, "peak_2000_prominence"] < limit_peak_2000[0]) | (
    pump_train_df.loc[:, "peak_2000_prominence"] > limit_peak_2000[1]
)
loc_train_variance = (pump_train_df.loc[:, "spectral_centroid_mean"] < limit_spectral_centroid_mean[0]) | (
    pump_train_df.loc[:, "spectral_centroid_mean"] > limit_spectral_centroid_mean[1]
)
a1 = np.sum(loc_train_mean & loc_train_variance)
a2 = np.sum(loc_train_mean & ~loc_train_variance)
a3 = np.sum(~loc_train_mean & loc_train_variance)

plt.subplot(121)
venn2(subsets=(a2, a3, a1), set_labels=("peak_2000_prominence", "spectral_centroid_mean"))
plt.title("Training dataset")


loc_test_mean = (pump_test_df.loc[:, "peak_2000_prominence"] < limit_peak_2000[0]) | (
    pump_test_df.loc[:, "peak_2000_prominence"] > limit_peak_2000[1]
)
loc_test_variance = (pump_test_df.loc[:, "spectral_centroid_mean"] < limit_spectral_centroid_mean[0]) | (
    pump_test_df.loc[:, "spectral_centroid_mean"] > limit_spectral_centroid_mean[1]
)
a1 = np.sum(loc_test_mean & loc_test_variance)
a2 = np.sum(loc_test_mean & ~loc_test_variance)
a3 = np.sum(~loc_test_mean & loc_test_variance)


plt.subplot(122)
venn2(subsets=(a2, a3, a1), set_labels=("peak_2000_prominence", "spectral_centroid_mean"))
plt.title("Test dataset")

In [ ]:
plt.figure(3, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Venn diagram between recordings considered abnormals by features 'peak_2000_prominence' and 'Variance'.",
    fontsize=20,
)

loc_train_mean = (pump_train_df.loc[:, "peak_2000_prominence"] < limit_peak_2000[0]) | (
    pump_train_df.loc[:, "peak_2000_prominence"] > limit_peak_2000[1]
)
loc_train_variance = (pump_train_df.loc[:, "spectral_bandwidth"] < limit_spectral_bandwidth[0]) | (
    pump_train_df.loc[:, "spectral_bandwidth"] > limit_spectral_bandwidth[1]
)
a1 = np.sum(loc_train_mean & loc_train_variance)
a2 = np.sum(loc_train_mean & ~loc_train_variance)
a3 = np.sum(~loc_train_mean & loc_train_variance)

plt.subplot(121)
venn2(subsets=(a2, a3, a1), set_labels=("peak_2000_prominence", "spectral_bandwidth"))
plt.title("Training dataset")


loc_test_mean = (pump_test_df.loc[:, "peak_2000_prominence"] < limit_peak_2000[0]) | (
    pump_test_df.loc[:, "peak_2000_prominence"] > limit_peak_2000[1]
)
loc_test_variance = (pump_test_df.loc[:, "spectral_bandwidth"] < limit_spectral_bandwidth[0]) | (
    pump_test_df.loc[:, "spectral_bandwidth"] > limit_spectral_bandwidth[1]
)
a1 = np.sum(loc_test_mean & loc_test_variance)
a2 = np.sum(loc_test_mean & ~loc_test_variance)
a3 = np.sum(~loc_test_mean & loc_test_variance)


plt.subplot(122)
venn2(subsets=(a2, a3, a1), set_labels=("peak_2000_prominence", "spectral_bandwidth"))
plt.title("Test dataset")

In [ ]:
plt.figure(3, figsize=(15, 8))
plt.clf()
plt.suptitle(
    "Venn diagram between recordings considered abnormals by features 'peak_2000_prominence' and 'Variance'.",
    fontsize=20,
)

loc_train_mean = (pump_train_df.loc[:, "spectral_centroid_mean"] < limit_spectral_centroid_mean[0]) | (
    pump_train_df.loc[:, "spectral_centroid_mean"] > limit_spectral_centroid_mean[1]
)
loc_train_variance = (pump_train_df.loc[:, "spectral_bandwidth"] < limit_spectral_bandwidth[0]) | (
    pump_train_df.loc[:, "spectral_bandwidth"] > limit_spectral_bandwidth[1]
)
a1 = np.sum(loc_train_mean & loc_train_variance)
a2 = np.sum(loc_train_mean & ~loc_train_variance)
a3 = np.sum(~loc_train_mean & loc_train_variance)

plt.subplot(121)
venn2(subsets=(a2, a3, a1), set_labels=("spectral_centroid_mean", "spectral_bandwidth"))
plt.title("Training dataset")


loc_test_mean = (pump_test_df.loc[:, "spectral_centroid_mean"] < limit_spectral_centroid_mean[0]) | (
    pump_test_df.loc[:, "spectral_centroid_mean"] > limit_spectral_centroid_mean[1]
)
loc_test_variance = (pump_test_df.loc[:, "spectral_bandwidth"] < limit_spectral_bandwidth[0]) | (
    pump_test_df.loc[:, "spectral_bandwidth"] > limit_spectral_bandwidth[1]
)
a1 = np.sum(loc_test_mean & loc_test_variance)
a2 = np.sum(loc_test_mean & ~loc_test_variance)
a3 = np.sum(~loc_test_mean & loc_test_variance)


plt.subplot(122)
venn2(subsets=(a2, a3, a1), set_labels=("spectral_centroid_mean", "spectral_bandwidth"))
plt.title("Test dataset")